# Register Cardi Data With Athena
This will create an Athena table in the Glue Catalog (Hive Metastore).

In [1]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
ingest_create_athena_table_cardio_passed = False

# Import PyAthena

In [3]:
from pyathena import connect

# Create Athena Table from Local cardio_train-checkpoint.csv and Quitline_Services_Available_Medications.csv Files

In [4]:

!aws s3 cp --recursive $s3_public_path_tsv/ $s3_private_path_tsv/
!aws s3 ls --recursive $s3_private_path_tsv


usage: aws s3 cp <LocalPath> <S3Uri> or <S3Uri> <LocalPath> or <S3Uri> <S3Uri>
Error: Invalid argument type
2025-03-16 05:44:59 aws-athena-query-results-786782285170-us-east-1
2025-03-07 06:28:33 cardiovale-solutions-datascience-pipeline
2025-02-27 06:09:40 sagemaker-studio-gyyb8wtvkop
2025-02-27 06:09:42 sagemaker-us-east-1-786782285170


In [5]:
# Set S3 staging directory
s3_staging_dir = "s3://cardiovale-solutions-datascience-pipeline/athena/staging/"

In [6]:
# Set Athena parameters
database_name = "cardiovale_db"
table_1_name = "create_table_cardio"
table_2_name = "create_table_quitline"

In [7]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [8]:
# Define SQL statement to drop table
statement = "DROP TABLE IF EXISTS cardiovale_db.cardio_train;"
pd.read_sql(statement, conn)

/tmp/ipykernel_127/1992382966.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [9]:
# SQL statement to execute for cardio table
statement_cardio = """CREATE EXTERNAL TABLE IF NOT EXISTS cardiovale_db.cardio_train (
    id INT,
    age INT,
    gender INT,
    height INT,
    weight FLOAT,
    ap_hi INT,
    ap_lo INT,
    cholesterol INT,
    gluc INT,
    smoke INT,
    alco INT,
    active INT,
    cardio INT
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n' 
STORED AS TEXTFILE
LOCATION 's3://cardiovale-solutions-datascience-pipeline/raw-data/'
TBLPROPERTIES ('skip.header.line.count'='1');
"""

print(statement_cardio)

CREATE EXTERNAL TABLE IF NOT EXISTS cardiovale_db.cardio_train (
    id INT,
    age INT,
    gender INT,
    height INT,
    weight FLOAT,
    ap_hi INT,
    ap_lo INT,
    cholesterol INT,
    gluc INT,
    smoke INT,
    alco INT,
    active INT,
    cardio INT
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '
' 
STORED AS TEXTFILE
LOCATION 's3://cardiovale-solutions-datascience-pipeline/raw-data/'
TBLPROPERTIES ('skip.header.line.count'='1');



In [10]:
# SQL statement to execute for quitline_services table
statement_quitline = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         state STRING,
         year INT,
         medication_available STRING,
         medication_type STRING
) ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\\n' 
LOCATION '{}' 
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, "quitline_services", "s3://cardiovale-solutions-datascience-pipeline/raw-data/Quitline_Services_Available_Medications_-_2010_To_Present_20250306.csv"
)

print(statement_quitline)


CREATE EXTERNAL TABLE IF NOT EXISTS cardiovale_db.quitline_services(
         state STRING,
         year INT,
         medication_available STRING,
         medication_type STRING
) ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n' 
LOCATION 's3://cardiovale-solutions-datascience-pipeline/raw-data/Quitline_Services_Available_Medications_-_2010_To_Present_20250306.csv' 
TBLPROPERTIES ('skip.header.line.count'='1')


In [15]:
import pandas as pd

pd.read_sql(statement_cardio, conn)

/tmp/ipykernel_127/1048700737.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_cardio, conn)


""


In [16]:
pd.read_sql(statement_quitline, conn)

/tmp/ipykernel_127/856595348.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_quitline, conn)


""


# Verify The Table Has Been Created Succesfully

In [17]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_127/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,_ipynb_checkpoints
1,cardio_train
2,cardio_train_csv
3,quitline___services_available___medications___...
4,quitline_services


In [18]:
if (table_1_name and table_2_name) in df_show.values:
    ingest_create_athena_table_cardio_passed = True

# Run A Sample Query for cardio table

In [19]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, "cardio_train_csv"
)

print(statement)


SELECT * FROM cardiovale_db.cardio_train_csv LIMIT 10


In [20]:
df = pd.read_sql(statement, conn)
df.head(5)

/tmp/ipykernel_127/2446512133.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio


In [22]:
!pip install awswrangler


  Using cached awswrangler-3.11.0-py3-none-any.whl.metadata (17 kB)
Using cached awswrangler-3.11.0-py3-none-any.whl (379 kB)


In [23]:
import awswrangler as wr

df = wr.s3.read_csv("s3://cardiovale-solutions-datascience-pipeline/raw-data/cardio_train.csv")
print(df.head())


2025-03-17 02:11:49,115	WARNING services.py:2022 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 1895804928 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.75gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-03-17 02:11:50,270	INFO worker.py:1786 -- Started a local Ray instance.


  id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio
0              0;18393;2;168;62.0;110;80;1;1;0;0;1;0                               
1              1;20228;1;156;85.0;140;90;3;1;0;0;1;1                               
2              2;18857;1;165;64.0;130;70;3;1;0;0;0;1                               
3             3;17623;2;169;82.0;150;100;1;1;0;0;1;1                               
4              4;17474;1;156;56.0;100;60;1;1;0;0;0;0                               


# Run A Sample Query for quitline table

In [24]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, "quitline___services_available___medications___2010_to_present_20250306_csv"
)

print(statement)


SELECT * FROM cardiovale_db.quitline___services_available___medications___2010_to_present_20250306_csv LIMIT 10


In [25]:
df = pd.read_sql(statement, conn)
df.head(5)

/tmp/ipykernel_127/2446512133.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,year,date,date_ref,locationabbr,locationdesc,topictype,topicdesc,measuredesc,sub-measure,variable,...,number_of_weeks_offered,limit_per_year,comments,geolocation,topictypeid,topicid,measureid,source,submeasureid,displayorder


In [27]:
import awswrangler as wr

df = wr.s3.read_csv("s3://cardiovale-solutions-datascience-pipeline/raw-data/Quitline___Services_Available___Medications_-_2010_To_Present_20250306.csv")
print(df.head())


   Year  Date Date_Ref LocationAbbr LocationDesc TopicType  \
0  2020    12  Jul-Dec           GA      Georgia  Quitline   
1  2020    12  Jul-Dec           MO     Missouri  Quitline   
2  2020    12  Jul-Dec           MT      Montana  Quitline   
3  2020    12  Jul-Dec           NJ   New Jersey  Quitline   
4  2020    12  Jul-Dec           OK     Oklahoma  Quitline   

            TopicDesc  MeasureDesc         Sub-Measure              Variable  \
0  Services Available  Medications    Nicotine Lozenge                   NaN   
1  Services Available  Medications    Nicotine Lozenge                   NaN   
2  Services Available  Medications  Bupropion (Zyban®)                   NaN   
3  Services Available  Medications    Nicotine Lozenge                   NaN   
4  Services Available  Medications        Nicotine Gum  All Eligible Callers   

   ... Number_of_Weeks_Offered    Limit_Per_Year Comments  \
0  ...                     NaN               NaN      NaN   
1  ...                  

*** SIGTERM received at time=1742177708 on cpu 0 ***
PC: @     0x7f3866128e2e  (unknown)  epoll_wait
    @     0x7f37fe1a64fd         64  absl::lts_20230802::AbslFailureSignalHandler()
    @     0x7f3866045520  (unknown)  (unknown)
[2025-03-17 02:15:08,662 E 127 127] logging.cc:440: *** SIGTERM received at time=1742177708 on cpu 0 ***
[2025-03-17 02:15:08,662 E 127 127] logging.cc:440: PC: @     0x7f3866128e2e  (unknown)  epoll_wait
[2025-03-17 02:15:08,665 E 127 127] logging.cc:440:     @     0x7f37fe1a6529         64  absl::lts_20230802::AbslFailureSignalHandler()
[2025-03-17 02:15:08,665 E 127 127] logging.cc:440:     @     0x7f3866045520  (unknown)  (unknown)


In [21]:
if not df.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

[OK]


# Review the New Athena Table in the Glue Catalog

In [22]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

/tmp/ipykernel_708/4130537117.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Store Variables for the Next Notebooks

In [23]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed                    -> True
ingest_create_athena_table_tsv_passed             -> True
s3_private_path_tsv                               -> 's3://sagemaker-us-east-1-786782285170/amazon-revi
s3_public_path_tsv                                -> 's3://usd-mads-508/amazon-reviews-pds/tsv'
setup_dependencies_passed                         -> True
setup_iam_roles_passed                            -> True
setup_instance_check_passed                       -> True
setup_s3_bucket_passed                            -> True


# Release Resources

In [24]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}